In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

<b><font size="6" color="red">ch14. 웹데이터수집</font></b>
# 1절. BeautifulSoup과 parser
`pip install bs4` 아나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함
- 공식 documentation : https://beautiful-soup-4.readthedocs.io/en/latest/

In [2]:
# 내 local PC의 파일을 url(http://~)처럼 접근
import requests # http 요청 처리하는 lib
from requests_file import FileAdapter # file://프로토콜을 다루기 위함

In [3]:
s = requests.Session() #  http 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
# file://경로로 들어오면 이 요청은 c:(로컬파일)을 접근
response = s.get('file:///ai/lecnote/01_python/data/ch14_sample.html')
response

<Response [200]>

In [4]:
if response:
    print('잘 접근하였습니다')
else:
    print('접근을 못 했습니다')

잘 접근하였습니다


In [5]:
response.status_code
    # 200 : 정상
    # 404 : 없는 페이지
    # 406 : get, post 오류

200

In [6]:
response.content # 바이너리 형식의 html내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [7]:
print(response.content.decode('utf-8'))

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
</head>
<body>
  <h1 class="greeting css" id="text">Hello, CSS</h1>
  <h1 class="css">Hi, CSS</h1>
  <div id="subject">subject 선택자 안의 내용</div>
  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>
  <div class="contents">
    선택자를 어떻게 작성하느냐에 따라
    <span>다른<b>요소가 반환</b></span>됩니다
  </div>
  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>
</body>
</html>


In [8]:
response.text # HTML파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [9]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, # response.text
                     "html.parser")
# soup

In [10]:
# soup.select_one('선택자') : 해당 선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :',el)
print('el.text :',el.text)
print('el.text :',el.string)
print('el의 속성들 :',el.attrs)
print('el의 id 속성 :',el.attrs['id']) # el.attrs은 딕셔너리
print('el의 id 속성 :',el.attrs.get('id'))
print('el의 href 속성 :',el.attrs.get('href'))
print('el의 태그이름 :',el.name)

el : <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text : Hello, CSS
el.text : Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text'}
el의 id 속성 : text
el의 id 속성 : text
el의 href 속성 : None
el의 태그이름 : h1


In [11]:
# soup.select('선택자') : 해당 선택자 모두
els = soup.select('h1') # h1태그를 list
print('els :',els)
print('els의 text들 :',[el.text for el in els])
print('els의 string들 :',[el.string for el in els])
print('els의 속성들 :',[el.attrs for el in els])
print('els의 class 속성들 :',[el.attrs.get('class') for el in els])

els : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들 : ['Hello, CSS', 'Hi, CSS']
els의 string들 : ['Hello, CSS', 'Hi, CSS']
els의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]
els의 class 속성들 : [['greeting', 'css'], ['css']]


In [12]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('selet_one:', soup.select_one('h1', {'class':'css'}) )
print('find:', soup.find('h1', {'class':'css'}) ) # soup.select_one('h1.css')
print('find:', soup.find('h1', class_='css') )
print()
print('select_one :', soup.select_one('h1#text') )
print('find :', soup.find('h1', {'id':'text'}))

selet_one: <h1 class="greeting css" id="text">Hello, CSS</h1>
find: <h1 class="greeting css" id="text">Hello, CSS</h1>
find: <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [13]:
# soup.font_all(태그, 속성) : soup.select('선택자')와 유사
print('select :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [14]:
print('select :', soup.select('h1.css,span'))
print('find :', soup.find_all('h1', {'class':'css'}) + \
                soup.find_all('span'))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [15]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a', {'class':'css'}) )
print('find(None) :', soup.find('a', {'class':'css'}) )

select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 BeautifulSoup을 활용한 html 웹데이터 수집
### 1)환율정보 가져오기 (네이버 증권=>시장지표)
- https://finance.naver.com/marketindex/

In [16]:
# 크롤링 허용범위는 사이트마다 ~/robots.txt에서 확인
    # Allow : / - 사이트의 모든 경로(/)에 대한 크롤링 허용

In [17]:
# 방법1
import requests 
from bs4 import BeautifulSoup
url ='https://finance.naver.com/marketindex/'
response = requests.get(url)
response
# response.status_code
# response.text/response.content.decode('cp949')
soup = BeautifulSoup(response.text,'html.parser')

In [18]:
# 방법2
from urllib.request import urlopen
url ='https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
# response.read() / response.read().decode('cp949')
soup = BeautifulSoup(response, 'html.parser')

In [19]:
# div.head_info > span.value (find함수)
prices = []
headinfos = soup.find_all('div', class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span',class_='value')
    # print(price.text.replace(',',''))
    prices.append(float(''.join(price.text.split(','))))
print(prices)

[1469.0, 949.24, 1702.42, 206.57, 154.88, 1.1573, 1.3096, 99.37, 58.49, 1708.98, 4213.6, 197702.39]


In [20]:
# div.head_info > span.value (select함수)
prices = soup.select('div.head_info > span.value')
print([float(p.text.replace(',','')) for p in prices])

[1469.0, 949.24, 1702.42, 206.57, 154.88, 1.1573, 1.3096, 99.37, 58.49, 1708.98, 4213.6, 197702.39]


In [21]:
titles = soup.select('h3.h_lst > span.blind')
for t in titles:
    print(t.text, end='\t')
# print([t.text+'\t' for t in titles])

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [22]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7,'') # 7번째에 ''추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [23]:
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')

하락	하락	하락	하락	상승	하락	하락	상승	하락	상승	상승	하락	

In [24]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [25]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text,
                                  prices[idx].text,
                                  units[idx],
                                 statuses[idx].text))

미국 USD : 1,469.00원 - 하락
일본 JPY(100엔) : 949.24원 - 하락
유럽연합 EUR : 1,702.42원 - 하락
중국 CNY : 206.57원 - 하락
달러/일본 엔 : 154.8800엔 - 상승
유로/달러 : 1.1573달러 - 하락
영국 파운드/달러 : 1.3096달러 - 하락
달러인덱스 : 99.3700 - 상승
WTI : 58.49달러 - 하락
휘발유 : 1708.98원 - 상승
국제 금 : 4213.6달러 - 상승
국내 금 : 197702.39원 - 하락


In [26]:
for title, price, unit, status in zip(titles,prices,units,statuses):
    print("{} : {}{} - {}".format(titles[idx].text,
                                  prices[idx].text,
                                  units[idx],
                                 statuses[idx].text))

국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락
국내 금 : 197702.39원 - 하락


### 2)이번주 로또번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin (google에 로또번호 당첨번호 검색)
```
    1197회(2025년 11월 08일 추첨)
    당첨번호 [1, 5, 7, 26, 28, 43]
    보너스 30
```

In [27]:
from urllib.request import urlopen
url ='https://dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
response.status
soup = BeautifulSoup(response, 'html.parser')

In [28]:
times = soup.select_one('div.win_result strong').text
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win  span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [29]:
print(times, date)
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

1197회 (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


### 3) 다음 검색 리스트
```
no title               href
0 "한풀 꺾인 비트코인 https://v.daum.net/v/20251110094711892
```

In [30]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# 방법2 
# from urllib.request import urlopen
# from urllib.parse import quote
# word = quote('비트코인')
# url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
# print(url)
# response = urlopen(url)
# soup = BeautifulSoup(response,"html.parser")

items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
item_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(item_el):
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                           'title':item.text,
                           'link':item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,"""비트코인, 닷컴버블과 판박이""…큰손들 매일 던진다 [비즈 나우]",http://v.daum.net/v/20251113075107107
1,1,셧다운 해제에도 식어버린 투심…비트코인 1억5300만원대 약세,http://v.daum.net/v/20251113092112374
2,2,셧다운 해제 기대감에 가상자산 약세...비트코인 10만1000달러,http://v.daum.net/v/20251113082848927
3,3,"“수능 끝, 금융 공부 ‘UP’”…업비트, 수험생에 아이패드·비트코인 쏜다",http://v.daum.net/v/20251113084508449
4,4,‘37조원 비트코인 베팅’ 흔들…기관투자자 이탈에 상승세 제동,http://v.daum.net/v/20251113021346163
5,5,1년새 1000% 폭등…'암호화 비트코인'이라 불리는 이 코인 [황두현의 웹3+],http://v.daum.net/v/20251112204650708
6,6,"써클, 매출·EPS 모두 예상 상회…USDC 유통 두 배 증가-[美증시 특징주]",http://v.daum.net/v/20251113082913938
7,7,"수고했'업'...업비트, 수험생 대상 퀴즈 이벤트",http://v.daum.net/v/20251113092140417
8,8,투자 광풍 풍자한 코인이 투기 상징 된 사연,http://v.daum.net/v/20251113090213338
9,9,"비트코인, 현물 ETF 유입 침체에 10만2000달러선 [크립토브리핑]",http://v.daum.net/v/20251112103550898


In [31]:
items_find_list = [] # 검색한 결과를 담을 리스트
item_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(item_el):
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append([idx, item.text, item.attrs['href']])
pd.DataFrame(items_find_list,columns=['no','title','link'])

,no,title,link
0,0,"""비트코인, 닷컴버블과 판박이""…큰손들 매일 던진다 [비즈 나우]",http://v.daum.net/v/20251113075107107
1,1,셧다운 해제에도 식어버린 투심…비트코인 1억5300만원대 약세,http://v.daum.net/v/20251113092112374
2,2,셧다운 해제 기대감에 가상자산 약세...비트코인 10만1000달러,http://v.daum.net/v/20251113082848927
3,3,"“수능 끝, 금융 공부 ‘UP’”…업비트, 수험생에 아이패드·비트코인 쏜다",http://v.daum.net/v/20251113084508449
4,4,‘37조원 비트코인 베팅’ 흔들…기관투자자 이탈에 상승세 제동,http://v.daum.net/v/20251113021346163
5,5,1년새 1000% 폭등…'암호화 비트코인'이라 불리는 이 코인 [황두현의 웹3+],http://v.daum.net/v/20251112204650708
6,6,"써클, 매출·EPS 모두 예상 상회…USDC 유통 두 배 증가-[美증시 특징주]",http://v.daum.net/v/20251113082913938
7,7,"수고했'업'...업비트, 수험생 대상 퀴즈 이벤트",http://v.daum.net/v/20251113092140417
8,8,투자 광풍 풍자한 코인이 투기 상징 된 사연,http://v.daum.net/v/20251113090213338
9,9,"비트코인, 현물 ETF 유입 침체에 10만2000달러선 [크립토브리핑]",http://v.daum.net/v/20251112103550898


In [32]:
items_find_list = [] # 검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    # print(idx, a.text, a.attrs['href'])
    # items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({'no':idx,
                           'title':a.text,
                           'link':a.attrs['href']})
pd.DataFrame(items_find_list,columns=['no','title','link'])

,no,title,link
0,0,"""비트코인, 닷컴버블과 판박이""…큰손들 매일 던진다 [비즈 나우]",http://v.daum.net/v/20251113075107107
1,1,셧다운 해제에도 식어버린 투심…비트코인 1억5300만원대 약세,http://v.daum.net/v/20251113092112374
2,2,셧다운 해제 기대감에 가상자산 약세...비트코인 10만1000달러,http://v.daum.net/v/20251113082848927
3,3,"“수능 끝, 금융 공부 ‘UP’”…업비트, 수험생에 아이패드·비트코인 쏜다",http://v.daum.net/v/20251113084508449
4,4,‘37조원 비트코인 베팅’ 흔들…기관투자자 이탈에 상승세 제동,http://v.daum.net/v/20251113021346163
5,5,1년새 1000% 폭등…'암호화 비트코인'이라 불리는 이 코인 [황두현의 웹3+],http://v.daum.net/v/20251112204650708
6,6,"써클, 매출·EPS 모두 예상 상회…USDC 유통 두 배 증가-[美증시 특징주]",http://v.daum.net/v/20251113082913938
7,7,"수고했'업'...업비트, 수험생 대상 퀴즈 이벤트",http://v.daum.net/v/20251113092140417
8,8,투자 광풍 풍자한 코인이 투기 상징 된 사연,http://v.daum.net/v/20251113090213338
9,9,"비트코인, 현물 ETF 유입 침체에 10만2000달러선 [크립토브리핑]",http://v.daum.net/v/20251112103550898


In [33]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    items_find_list = [] # 검색한 결과를 담는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [34]:
collect_news_list('AI', 1)

[[0,
  ' AI시대의 그늘, ‘AI 문해력 격차’… “민관 합동 교육으로 불평등 줄여야” ',
  'http://v.daum.net/v/20251112002849967'],
 [1, ' 김부장이 송과장·정대리보다 AI 더 열심히 쓴다 ', 'http://v.daum.net/v/20251112084738042'],
 [2,
  ' 대한민국 최초 AI국 신설 김동연 “AI위원회, 경계와 한계 없이 활동” 요청 ',
  'http://v.daum.net/v/20251112090347712'],
 [3,
  ' 뤼튼 이세영 대표 "AI 격변기, 민관이 함께 AI리터러시 교육 나서야" ',
  'http://v.daum.net/v/20251111135631294'],
 [4,
  ' 카카오뱅크, 업계 최초 ‘AI 품질관리 프레임워크’ 도입…AI 리스크 관리 본격 ',
  'http://v.daum.net/v/20251113091027744'],
 [5,
  ' 수자원공사, 상수도 AI 전환 심포지엄 개최…AI 물 산업 등 논의 ',
  'http://v.daum.net/v/20251113082025678'],
 [6,
  " 강기정 시장, 하정우 AI수석과 '광주 AI실증도시' 논의 ",
  'http://v.daum.net/v/20251113083047973'],
 [7,
  ' 뜨거운 AI 주도권 경쟁… 손정의, 엔비디아株 팔고 ‘오픈AI’ 올인 ',
  'http://v.daum.net/v/20251113030328243'],
 [8,
  ' “AI가 식당 대기식당 예측”…삼성웰스토리, AI솔루션 도입 ',
  'http://v.daum.net/v/20251113080139296'],
 [9,
  ' 에이전틱 AI와 피지컬AI, 내년 생산성 혁명 이끌 것 ',
  'http://v.daum.net/v/20251113060216500']]

In [35]:
import time
keywords =['청바지','동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결과를 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어 {keyword} 검색결과 수집중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword, page))
        elif i==1:
            result1.extend(collect_news_list(keyword, page))
        time.sleep(0.5)

===1번째 검색어 청바지 검색결과 수집중입니다===
===2번째 검색어 동대문 검색결과 수집중입니다===


In [36]:
result0_df = pd.DataFrame(result0, columns=['no', 'title', 'link'])
result1_df = pd.DataFrame(result1, columns=['no', 'title', 'link'])
result1_df.head()

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 ‘핫플’ 됐다",http://v.daum.net/v/20251111111251709
1,1,"동대문구, 사가정로 안전보행로 개통… 주민 숙원 20년 만에 해소",http://v.daum.net/v/20251113073513911
2,2,동대문구 5번째 서울형 키즈카페 '새샘공원점' 개관,http://v.daum.net/v/20251113070516516
3,3,"음주운전 차, 동대문역서 인도 돌진...日 관광객 '사망'",http://v.daum.net/v/20251103073316480
4,4,"노보텔 앰배서더 동대문, 연말 맞아 '페스티브 파티’ 패키지 출시",http://v.daum.net/v/20251111155426418


In [37]:
result0_df.to_csv(f'data/ch14_{keyword[0]}.csv',index=False)
result1_df.to_csv(f'data/ch14_{keyword[1]}.csv',index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라우저를 통해 요청하는 것처럼 보이기

In [38]:
# 방법1 
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인') # 한글을 encoding 처리
url ='https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = urlopen(url)
response.status
soup = BeautifulSoup(response, 'html.parser')
soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


<script type="text/javascript"> location.replace("https://captcha.search.daum.net/show?redirect=https%3A%2F%2Fsearch.daum.net%2Fsearch%3Fw%3Dnews%26nil_search%3Dbtn%26DA%3DPGD%26enc%3Dutf8%26cluster%3Dy%26cluster_page%3D1%26q%3D%25EB%25B9%2584%25ED%258A%25B8%25EC%25BD%2594%25EC%259D%25B8"); </script>

In [39]:
# 방법 2: 
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인') # 한글을 encoding 처리
url ='https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
# headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'}
# request = Request(url, headers=headers)
request = Request(url)
request.add_header('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36')
response=urlopen(request)
soup = BeautifulSoup(response,'html.parser')
# soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


- https://www.melon.com/chart/index.htm
    - https://www.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [40]:
# 방법1
import requests
url = 'https://www.melon.com/chart/index.htm'
melonpage = requests.get(url)
melonpage
# soup = BeautifulSoup(melonpage.text , 'html.parser')
# print(soup)

<Response [406]>

In [41]:
# 방법2
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
# melonpage = urlopen(url) # 에러남
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup =BeautifulSoup(melonpage, 'html.parser')
# soup

In [42]:
# 방법1 : User-Agent 추가
url = 'https://www.melon.com/chart/index.htm'
# melonpage = urlopen(url) # 에러남
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"}
melonpage = requests.get(url,headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                     "html.parser")

ranks = soup.select('td div.wrap > span.rank')
titles = soup.select('div.ellipsis.rank01 > span') # title.text.strip()
singers = soup.select('div.ellipsis.rank02') # singer.text.strip()[:20]
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXXNMIXX
2위 | 타임캡슐 - 다비치다비치
3위 | Golden - HUNTR/X, EJAE, AUDRE
4위 | 달리 표현할 수 없어요 - 로이킴로이킴
5위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
6위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
7위 | Drowning - WOODZWOODZ
8위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9위 | Soda Pop - KPop Demon Hunters C
10위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
11위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
12위 | FAMOUS - ALLDAY PROJECTALLDAY
13위 | IRIS OUT - Kenshi YonezuKenshi 
14위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
15위 | XOXZ - IVE (아이브)IVE (아이브)
16위 | 한번 더 이별 - 이창섭이창섭
17위 | Rich Man - aespaaespa
18위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
19위 | 너에게 닿기를 - 10CM10CM
20위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
21위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
22위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23위 | body - 다영 (DAYOUNG)다영 (DAYO
24위 | Whiplash - aespaaespa
25위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
26위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
27위 | 청춘만화 - 이무진이무진
28위 | 천상연 - 이창섭이창섭
29위 | 멸종위기사랑 - 이찬혁이찬혁
30위 | 어떻게 이별까지 사

### 5) 네이버 지식인으로 검색(open API 사용x)
- 특정 keyword를 특정페이지 수만큼

In [43]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '챗지피티'
# url = f'https://kin.naver.com/search/list.naver' 
# params = {'query':keyword}
# response = get(url, params=params)
url = f'https://kin.naver.com/search/list.naver?query={keyword}' 
response = get(url)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')
# soup

200


In [44]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
keyword = quote('챗지피티')
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
print(url)
response = urlopen(url)
print(response.status)
soup = BeautifulSoup(response, 'html.parser')
# soup

https://kin.naver.com/search/list.naver?query=%EC%B1%97%EC%A7%80%ED%94%BC%ED%8B%B0
200


In [45]:
# 페이징 포함
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('챗지피티')
pages = 3
items_list = [] # 크롤링한 데이터를 담을 list(title, link)
for page in range(1, pages+1):
    url=f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = urlopen(url)
    # print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    items = soup.select('dt > a')
    for item in items:
        title = item.text
        link = item.attrs.get('href')
        items_list.append({'title':title,
                           'link':link
                          })
df= pd.DataFrame(items_list)
print(df.shape)
print(df.loc[29, "link"])
df.head()

(30, 2)
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1060101&docId=490138216&qb=7LGX7KeA7ZS87Yuw&enc=utf8


,title,link
0,챗지피티 관련 질문,https://kin.naver.com/qna/detail.naver?d1id=1&...
1,챗지피티 자동 결제 안 됨,https://kin.naver.com/qna/detail.naver?d1id=1&...
2,챗지피티 쉐어,https://kin.naver.com/qna/detail.naver?d1id=1&...
3,챗지피티 전송 오류,https://kin.naver.com/qna/detail.naver?d1id=1&...
4,Ai 챗지피티 답변,https://kin.naver.com/qna/detail.naver?d1id=1&...


## 2.2 openAPI사용 : json 웹데이터 수집
### 1) 네이버 지식인으로 검색(openAPI 사용o)
- 네이버 개발자센터에서 애플리케이션등록(이름/검색/web설정 http://localhost)
- .env파일에 CLIENT_ID/CLIENT_SECRET 환경변수 저장
- 환경 변수를 읽기 위해 `pip install dotenv`
- 특정 keyword를 지식 검색(데이터수 30개)

In [46]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [47]:
# 환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv(
    # dotenv_path = '.env'
            ) # 현 소스와 같은 폴더내의 .env를 메모리에 load
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))

True

In [48]:
# 방법2
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("챗지피티")
url = "https://openapi.naver.com/v1/search/kin?query=" + encText # JSON 결과

# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
response = urllib.request.Request(url, headers=headers)

response = urllib.request.urlopen(request)

rescode = response.status
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    print(response_body.decode('utf-8')[:50])
else:
    print("Error Code:" + rescode)

<class 'str'>
<!DOCTYPE html>
<html lang="ko">










In [49]:
# 방법1
import requests
import pandas as pd
import json # response텍스트를 json스타일의 딕셔너리로
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '쳇지피티'
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f'https://openapi.naver.com/v1/search/kin?query={query}&display=30'
# response = requests.get(url, headers=headers)
url = 'https://openapi.naver.com/v1/search/kin'
params = {'query':query, 'display':30}
response = requests.get(url, params=params, headers=headers)
# print(response.text[:500])
# items = json.loads(response.text)['items']
items = response.json()['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list, columns=['title', 'link', 'description']).head()

,title,link,description
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...,쳇지피티 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요...
1,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티한테 질문 제대로 하려면 어떻게 해야 하냐요?... 쳇지피티 한테 하는질문도...
2,쳇지피티 질문,https://kin.naver.com/qna/detail.naver?d1id=1&...,제가 쳇지피티 한테 물어볼게 있어서 지피티야 이러면서 운을 뗐는데 애가 갑자기 제 ...
3,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지) 남매입니다 남자 1986년 9...
4,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료버전으로 카톡대화창 전송해서 상대의심리에 관한 분석요청을 많이했는데 첨...


### quiz)네이버 openAPI를 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지.csv출력
```
    제목,링크,썸네일,sizeheight,sizewidth
```

In [50]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법2)'
    from urllib.request import urlopen, Request
    from urllib.parse import quote
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    encText = quote(query)
    url = f'https://openapi.naver.com/v1/search/image?query={encText}&display=100'
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    request = Request(url, headers=headers)
    response = urlopen(request)
    # print(response.read().decode('utf-8'))
    items = json.loads(response.read().decode('utf-8'))['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

,제목,링크,썸네일,sizeheight,sizewidth
65,남자 스탠다드 캣 워싱 와이드 데님팬츠 청바지 : 옴므티디,http://shop1.phinf.naver.net/20251017_1/176068...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


In [51]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    #print(items[0])
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth'))
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

,제목,링크,썸네일,sizeheight,sizewidth
68,봄 가을 남성청바지 와이드 캐주얼팬츠 데님바지 작업복 데님팬츠 남자바지 28 902,http://shopping.phinf.naver.net/main_4736873/4...,https://search.pstatic.net/common/?type=b150&s...,790,790


In [52]:
print(df.loc[0, '링크'])
print(df.loc[0, '썸네일'])

https://shop-phinf.pstatic.net/20251015_281/1760512390478oRzer_PNG/42211957144911892_1321802660.png
https://search.pstatic.net/common/?type=b150&src=https://shop-phinf.pstatic.net/20251015_281/1760512390478oRzer_PNG/42211957144911892_1321802660.png


In [53]:
def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query.확장자로 local에 저장'
    file_extension = link.split('.')[-1] # 확장자
    quote_index = file_extension.find("?") # 확장자뒤에 ?가 위치 .jpg?w=780
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
    img = requests.get(link).content # 바이너리
    with open(f'img/{attr}_{idx+1}_{query}.{file_extension}', 'wb') as f:
        f.write(img)
save_image('메인', 0, df.loc[0, '링크'], '청바지')

In [54]:
def get_image_list_save_image(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    #print(items[0])
    items_list = []
    for idx, item in enumerate(items):
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth'))
        })
 #이미지 파일 저장
        save_image('메인', idx, link, query)
        save_image('썸네일', idx, thumbnail, query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행중 = = =')
    print('= = = 완료 = = =')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('img/ch14_청바지.csv')
df.sample()

= = = 20% 진행중 = = =
= = = 40% 진행중 = = =
= = = 60% 진행중 = = =


ReadTimeout: HTTPSConnectionPool(host='search.pstatic.net', port=443): Read timed out. (read timeout=None)

## 2.3 XML 웹데이터 수집
- RSS / open API을 통해 XML 웹데이터 수집
### 1) 전국날씨 RSS를 BeautifulSoup을 이용한 크롤링
- 구글에 "기상청 RSS" 검색한 첫번째 사이트 이동

In [55]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
items_list = []
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'
# 방법1
# target = requests.get(url)
# soup = BeautifulSoup(target.text, "xml") # pip install lxml 필요없음
# 방법2
target = urlopen(url)
soup = BeautifulSoup(target, "xml")
local_tas = soup.select('local_ta')
for local_ta in local_tas:
    local_name = local_ta.select_one('local_ta_name').text
    week1_normalYear = local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local_ta.select_one('week1_local_ta_similarRange').text
    week1_minVal = local_ta.select_one('week1_local_ta_minVal').text
    week1_similarVal = local_ta.select_one('week1_local_ta_similarVal').text
    week1_maxVal = local_ta.select_one('week1_local_ta_maxVal').text
    items_list.append({
        '지역':local_name,
        '1주평년기온':week1_normalYear,
        '1주기온범위':week1_similarRange,
        '1주낮을확률':week1_minVal,
        '1주비슷할확률':week1_similarVal,
        '1주높을확률':week1_maxVal
    })
df = pd.DataFrame(items_list)
df['1주평년기온']= df['1주평년기온'].astype(np.float64)

### 2) XML응답하는 open API활용
- data.go.kr에서
    - 서울특별시_노선정보조회 서비스(버스ID, 정류장목록과정류장ID)
    - 서울특별시_버스위치정보조회 서비스(실시간 버스 위치 목록)

In [56]:
# step 1 : 버스번호의 버스ID 받아오기
# 서울특별시_노선정보조회 서비스 - 3번 기능(getBusRouteList) 이용

In [57]:
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))

True

In [58]:
from bs4 import BeautifulSoup
# urlretrieve(url, 저장경로) : url의 파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote
# busNum = quote('구로09')
busNum = '6512'
key = os.getenv('KEY')
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
print(url)
savefilename1='data/ch14_busInfo.xml'
urlretrieve(url, savefilename1) # xml파일(url)을 savefilename1로 저장
with open(savefilename1, encoding='utf-8') as f:
    xml = f.read();
soup = BeautifulSoup(xml, 'xml')

http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey=fbf9740d57c6f3864c800cc458132bf60fb9a7a3f875245823d06b006dfe7167&strSrch=6512


In [59]:
import requests
busNum = '6512'
key = os.getenv('KEY')
url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
respose = requests.get(url1)
soup = BeautifulSoup(respose.text,'xml')

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        break;
print('busRouteId =', busRouteId)

busRouteId = 100100292


In [60]:
# step2. 버스id로 버스정류장목록받아오기 (정류장명, id, 경도, 위도)
# 서울특별시_노선정보조회 서비스 - 4번 기능(getStaionsByRouteList)

In [61]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
response = requests.get(url2)
soup = BeautifulSoup(response.content, 'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수 : {len(itemLists)}')
bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text # 정류소 이름
    station   = itemList.find('station').text   # 정류소 id
    gpsX      = itemList.find('gpsX').text      # 경도
    gpsY      = itemList.find('gpsY').text      # 위도
    bus_station.append([stationNm,station,gpsX,gpsY])
df_station = pd.DataFrame(bus_station, columns=['정류소명','id','경도','위도'])
df_station

6512번 정류장 갯수 : 81


,정류소명,id,경도,위도
0,구로두산아파트,116000032,126.890072,37.488807
1,남구로역6번출구,116000158,126.888568,37.487735
2,남구로역5번출구.구로4동자치회관,116000035,126.886453,37.48531
3,남구로역,116000048,126.8856450789,37.4860984011
4,구로시장,116000049,126.88486,37.487987
...,...,...,...,...
76,구로구청,116000026,126.8876042378,37.4947508763
77,구로4동우체국.고대구로병원정문,116000052,126.8840651203,37.4899022236
78,구로시장,116000050,126.88461,37.48798
79,남구로역2번출구,116000034,126.886787,37.485278


In [62]:
# step3. 버스id로 실시간위치정보를 받아오기(차량번호, 혼잡도, 최종정류장id, 다음정류장id, 도착소요시간)
# 서울특별시_버스위치정보조회 서비스 - 2번 (getBusPosByRtidList)

In [63]:
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey={key}&busRouteId={busRouteId}'
# print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text, 'xml')
itemLists = soup.select('itemList')
print(f'운행중인 버스는 {len(itemLists)}대입니다')
bus_position = []
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text # 차량번호
    congetion = itemList.select_one('congetion').texy # 혼잡도(congetion)
     # 0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡
    congetion = '없음' if congetion=='0' \
        else '여유' if congetion=='3' \
        else '보통' if congetion=='4' \
        else '혼잡' if congetion=='5' else '매우혼잡'
    bus_position.append({
        '차량번호':plainNo,
        '혼잡도':congetion
    })
df_position = pd.DataFrame(bus_position)
df_position

운행중인 버스는 19대입니다


,차량번호,혼잡도
0,서울74사1122,매우혼잡
1,서울74사2757,매우혼잡
2,서울74사3375,매우혼잡
3,서울74사3371,매우혼잡
4,서울75사2791,매우혼잡
5,서울74사2832,매우혼잡
6,서울74사2709,매우혼잡
7,서울74사2750,매우혼잡
8,서울74사3374,매우혼잡
9,서울75사2772,매우혼잡


In [64]:
df_station.loc[df_station['id']=='100100292','정류소명'].iloc[0]

IndexError: single positional indexer is out-of-bounds

In [ ]:
df_station.loc[df_station['id']=='107000079','정류소명'].iloc[0]

In [ ]:
def station_name(row):
    row = row.copy()
    'row의 최종정류소명과 다음정류소명을 추가하고, 도착소요시간을 분으로 바꿔 return'
    row['이전정류소명'] = df_station.loc[df_station['id']==row['최종정류소id'],'정류소명'].iloc[0]
    row['다음정류소명'] = df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    minite = int(row['도착소요시간'])//60 # //몫연산자
    sec    = int(row['도착소요시간']) % 60
    row['도착소요시간'] = f'{minite}분{sec}초'
    return row
#station_name(df_position.loc[20])

# 3절 연습문제(Quiz1)
- yes24의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장하세요.
    - 베스트셀러 정보 수집 주소 : http://www.yes24.com/24/category/bestseller (1페이지만 추출)
    - https://www.yes24.com/product/category/bestseller?pageNumber={page} (여러 페이지 추출)
ch14_yes24.csv나 ch14_yes24.txt의 내용(구분문자를 ,나 \t,....)
    - 순위,책이름,저자,출판사,가격
      1,[도서]트렌드코리아2026,"김난도,전미영,최지혜,권정윤,한다혜, 미래의창",18,000원
      24,[만화]그 비스트 어쩌구,후쿠다신이치글그림/박연지역, 소미미디어,25,200원

In [68]:
# import
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [69]:
# 방법1
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.strip() for authors_el in authors_els]
    # authors = [authors_el.get_text(strip=True) for authors_el in authors_els]
#     print(authors)
    for i, author in enumerate(authors):
        match = re.search(r"(.*)외 \d+명", author)
        if(match):
            authors[i] = match.group()
            print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

1 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름
11 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름


,순위,책이름,저자,출판사,가격
0,1,처음 만나는 양자의 세계,채은미 저,북플레저,"17,100원"


In [70]:
# 방법2
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = urlopen(url)
    soup = BeautifulSoup(response, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.split('정보 더 보기/감추기')[0].strip() 
                                   for authors_el in authors_els]
#     for i, author in enumerate(authors):
#         match = re.search(r"(.*)외 \d+명", author)
#         if(match):
#             authors[i] = match.group()
#             print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

,순위,책이름,저자,출판사,가격
0,1,처음 만나는 양자의 세계,채은미 저,북플레저,"17,100원"
